In [1]:
!nvidia-smi

Sat Aug  5 12:25:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install torch
!pip install bitsandbytes
!pip install accelerate@git+https://github.com/huggingface/accelerate.git
!pip install transformers@git+https://github.com/huggingface/transformers.git
!pip install peft@git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.4 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-install-wn3emles/accelerate_39f83af29d6e4170bab07ed71d793f0f
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-install-wn3emles/accelerate_39f83af29d6e4170bab07ed71d793f0f
  Resolved https://github.com/huggingface/accelerate.git to commit 30eff605df5db9ba20289a0c7471e4d8547a5a8f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.22.0.dev0-py3-none-any.whl size=249339 sha256=60b28d5089790616e148def474bb473e28f140155c176465182d8892b65ddd71
  Stored in directory: /tmp/pip-ephem-wheel-cache-74t3y72p/wheels/9c/a3/1e/47368f9b6575655fe9ee1b6350cfa7d4b0befe66a35f8a8365
Successfully built accelerate
  Cloning https://github.com/huggingface/transformers.g

In [ ]:
# Uncomment if necessary
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [1]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [7]:
!pip install sentencepiece
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895239 sha256=6b7e03bd72073efa7e69ff3145882cd35d338a4b831d1d50778c3e4efb046022
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [2]:
from transformers import MarianMTModel, MarianTokenizer

In [3]:
target_language_suffix = ">>pt<< "

model_name = "Helsinki-NLP/opus-mt-en-ROMANCE"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to('cuda')

In [4]:
import torch

max_token_lenght = 475

def marian_translate(src_text):
  len_error = False

  try:
    tokenized_input = tokenizer(target_language_suffix + src_text, return_tensors="pt", padding=True).to('cuda')
    token_lengths = [len(sequence) for sequence in tokenized_input['input_ids']]
    max_length = max(token_lengths)
    if max_length <= max_token_lenght:
      translated = model.generate(**tokenized_input, max_new_tokens=512)
    else:
      len_error = True
  except:
      raise Exception("This is a fatal error! (CUDA)")

  if len_error:
    return "LEN_ERROR"
  else:
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated][0]

In [5]:
# Test

src_text = "This is a test."
print(marian_translate(src_text))

Isto é um teste.


In [6]:
import pandas as pd

model_code = 'marianmt'

try:
  opus_100_pt_validation = pd.read_parquet('./opus_100_pt_validation_' + model_code + '.parquet')
except:
  opus_100_pt_validation = pd.read_parquet('https://huggingface.co/api/datasets/opus100/parquet/en-pt/validation/0.parquet')

In [ ]:
dataset = opus_100_pt_validation.translation

row_number = len(dataset)
current_row = 0

for json in opus_100_pt_validation.translation:
    current_row=current_row+1

    if model_code in json and json[model_code]:
        print(str(current_row)+ "/" + str(row_number)+ " - Tradução existente: " + json[model_code] )
    else:
        originalString = json['en']
        traducao = marian_translate(originalString)
        print(str(current_row)+ "/" + str(row_number)+ " - Tradução: " + traducao)
        json[model_code]=traducao
        opus_100_pt_validation.to_parquet('./opus_100_pt_validation_' + model_code + '.parquet')


1/2000 - Tradução: Ele nunca fez nada.
2/2000 - Tradução: 50/50 tiros?
3/2000 - Tradução: Sim, e vou deixar-te ir buscar a tua oportunidade, mas vais ficar bem?
4/2000 - Tradução: Foi culpa tua, não foi?
5/2000 - Tradução: Então vai lá!
6/2000 - Tradução: Porque ainda não o lavaste.
7/2000 - Tradução: Parecia a voz do Parker ao telefone?
8/2000 - Tradução: Deixe-me dizer-lhe o que o Detective Bird acha que aconteceu.
9/2000 - Tradução: Se me amasses, terias remado.
10/2000 - Tradução: - Temos dois suspeitos.
11/2000 - Tradução: - Abrimos a porta...
12/2000 - Tradução: Os empréstimos podem atrair uma bonificação de juros a título do orçamento operacional.
13/2000 - Tradução: Então, o Mikey Molloy fingiu ter um dos seus ataques... para que eu pudesse entrar quando ninguém olhasse.
14/2000 - Tradução: Juro-te que este desafio será resolvido.
15/2000 - Tradução: O Mike estava à tua procura.
16/2000 - Tradução: Mas não somos exterminados com um botão.
17/2000 - Tradução: Passei por muitos f

In [ ]:

saved_parquet = pd.read_parquet('./opus_100_pt_validation_' + model_code + '.parquet')
print(saved_parquet.translation[200])